# **Redes Neurais**
## **Modelo 3 - Variational Autoencoder (VAE)**

- Aluno: Tales Miguel
- RA: 140247

- Professor Dr. Marcos G. Quiles

## **Instruções**

▸Selecionar 2 datasets (rotulados) -----> Se for um relatório detalhado, pode usar apenas 1 dataset

▸Treinar modelos VAEs:
  - Ajustar o melhor modelo (topologia) segundo a função de custo (conjunto validação)

▸Explorar o espaço latente:
  - Gerar gráficos com a projeção do espaço latente em 2D (PCA)
  - Usar os rótulos na projeção

▸Algumas questões:
  1. Há formação de clusters no espaço latente?
  2. Há separação dos rótulos no espaço latente?
  3. A projeção ilustra quanto da variância?

▸ Adicional (opcional): Enviesar a formação do espaço latente com os exemplos rotulados

## **1. Escolha do Dataset e Pré-processamento dos dados**

### **1.1. Dataset Escolhido - EMNIST Letters**

**Dataset**: EMNIST Letters  
**Fonte**: Extended MNIST (Cohen et al., 2017) | [Acesso via TensorFlow Datasets](https://www.tensorflow.org/datasets/catalog/emnist)

**Características:**
- **Samples**: ~145.000 imagens (124.800 treino + 20.800 teste)
- **Dimensões**: 28x28 pixels (grayscale)
- **Features**: 784 (28×28 pixels normalizados)
- **Classes**: 26 letras maiúsculas (A-Z)

### **1.2. Imports e Pré-processamento**

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from sklearn.metrics import silhouette_score, davies_bouldin_score, calinski_harabasz_score

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, Model
from tensorflow.keras.optimizers import Adam

import tensorflow_datasets as tfds

np.random.seed(42)
tf.random.set_seed(42)

2025-11-06 18:26:18.519349: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-11-06 18:26:18.533126: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1762464378.545940   20537 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1762464378.548639   20537 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1762464378.554105   20537 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking 

ImportError: cannot import name 'runtime_version' from 'google.protobuf' (/home/tales/miniconda3/envs/tf-gpu/lib/python3.11/site-packages/google/protobuf/__init__.py)

In [ ]:
dataset, info = tfds.load('emnist/letters', split=['train', 'test'], as_supervised=True, with_info=True)
train_dataset, test_dataset = dataset

x_train_list, y_train_list = [], []
for image, label in tfds.as_numpy(train_dataset):
    x_train_list.append(image)
    y_train_list.append(label)

x_test_list, y_test_list = [], []
for image, label in tfds.as_numpy(test_dataset):
    x_test_list.append(image)
    y_test_list.append(label)

x_train = np.array(x_train_list)
y_train = np.array(y_train_list)
x_test = np.array(x_test_list)
y_test = np.array(y_test_list)

x_train = x_train.astype('float32') / 255.0
x_test = x_test.astype('float32') / 255.0

x_train = x_train.reshape(-1, 28, 28)
x_test = x_test.reshape(-1, 28, 28)

y_train = y_train - 1
y_test = y_test - 1

x_train, x_val, y_train, y_val = train_test_split(
    x_train, y_train, test_size=0.2, random_state=42, stratify=y_train
)

print(f"Train: {x_train.shape}, Val: {x_val.shape}, Test: {x_test.shape}")
print(f"Pixel range: [{x_train.min():.1f}, {x_train.max():.1f}]")
print(f"Classes: {np.unique(y_train)} (0=A, 25=Z)")
print(f"Total de samples: {len(x_train) + len(x_val) + len(x_test)}")

### **1.3. Visualização Exploratória dos Dados**

In [ ]:
letter_names = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M',
                'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z']

fig, axes = plt.subplots(3, 10, figsize=(20, 6))
axes = axes.ravel()

for i in range(26):
    idx = np.where(y_train == i)[0][0]
    img = x_train[idx]
    axes[i].imshow(img, cmap='gray')
    axes[i].set_title(f'{letter_names[i]}', fontsize=12, fontweight='bold')
    axes[i].axis('off')

for i in range(26, 30):
    axes[i].axis('off')

plt.suptitle('Amostras do Dataset EMNIST Letters (A-Z)', fontsize=16, fontweight='bold')
plt.tight_layout()
plt.show()

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

unique, counts = np.unique(y_train, return_counts=True)
axes[0].bar([letter_names[i] for i in unique], counts, color='skyblue', edgecolor='black')
axes[0].set_title('Distribuição das Classes (Conjunto de Treino)', fontsize=13, fontweight='bold')
axes[0].set_xlabel('Letra', fontsize=11)
axes[0].set_ylabel('Frequência', fontsize=11)
axes[0].grid(True, alpha=0.3, axis='y')
axes[0].tick_params(axis='x', rotation=0)

total_samples = len(x_train) + len(x_val) + len(x_test)
train_pct = len(x_train) / total_samples * 100
val_pct = len(x_val) / total_samples * 100
test_pct = len(x_test) / total_samples * 100

axes[1].barh(['Dataset'], [train_pct], left=0, label=f'Train ({train_pct:.1f}%)', color='skyblue')
axes[1].barh(['Dataset'], [val_pct], left=train_pct, label=f'Val ({val_pct:.1f}%)', color='orange')
axes[1].barh(['Dataset'], [test_pct], left=train_pct+val_pct, label=f'Test ({test_pct:.1f}%)', color='lightcoral')
axes[1].set_xlabel('Porcentagem (%)', fontsize=11)
axes[1].set_title('Divisão Train/Val/Test', fontsize=13, fontweight='bold')
axes[1].legend()
axes[1].set_xlim(0, 100)

plt.tight_layout()
plt.show()

## **2. Implementação do Variational Autoencoder (VAE)**

### **2.1. Configuração das Topologias para Experimentos**

### **2.2. Treinamento e Seleção da Melhor Topologia**

### **2.3. Análise Comparativa das Topologias**

## **3. Exploração do Espaço Latente**

### **3.1. Projeção do Espaço Latente em 2D (PCA)**

### **3.2. Questão 1: Há Formação de Clusters no Espaço Latente?**

### **3.3. Questão 2: Há Separação dos Rótulos no Espaço Latente?**

### **3.4. Questão 3: A Projeção Ilustra Quanto da Variância?**

### **3.5. Interpolação no Espaço Latente**

### **3.6. Geração de Novas Amostras**

## **4. Avaliação Final e Conclusões**

### **4.1. Avaliação no Conjunto de Teste**

### **4.2. Síntese dos Resultados e Conclusões**